# Exact Numerical Calculation of $n_s$ and $r$

This notebook calculates the **Scalar Spectral Index ($n_s$)** and the **Tensor-to-Scalar Ratio ($r$)** by solving the exact Mukhanov-Sasaki equations. 

Unlike the slow-roll approximation, which assumes parameters like $\epsilon$ and $\eta$ are small and constant, this method involves:
1.  Solving the **exact background dynamics** ($a(t), \phi(t), H(t)$).
2.  Finding the exact moment of **horizon crossing** for the pivot scale $k_*$.
3.  Solving the **Mukhanov-Sasaki mode equations** for $k_*$ and its neighbors to determine the precise power spectrum $\mathcal{P}_{\mathcal{R}}(k)$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import json
import datetime
import uuid

# Import modules from the current directory (project root)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from inflation_models import HiggsModel, QuadraticModel
import inf_dyn_background as bg_solver
import inf_dyn_MS_full as ms_solver

## 1. Theory and Nomenclature

### Background Dynamics
We solve the coupled system for the scalar field $\phi$ and the scale factor $a$ (in suitable dimensionless units):

$$ \ddot{\phi} + 3H\dot{\phi} + V'(\phi) = 0 $$
$$ H^2 = \frac{1}{3M_{pl}^2} \left( \frac{1}{2}\dot{\phi}^2 + V(\phi) \right) $$

### Perturbation Eequations
We obtain the power spectra by evolving the **Mukhanov-Sasaki variables** for scalars ($v_k$) and tensors ($h_k$). 

The scalar equation is:
$$ v_k'' + \left( k^2 - \frac{z''}{z} \right) v_k = 0, \quad \text{where } z = \frac{a \dot{\phi}}{H} $$

And for tensors:
$$ h_k'' + \left( k^2 - \frac{a''}{a} \right) h_k = 0 $$

### Code Nomenclature & Units

The simulation works in dimensionless units scaled by the Planck mass $M_{pl}$ and a time-scaling parameter $S$.  

**Core Parameters:**
- **$M_{pl}$**: Reduced Planck Mass ($2.435 \times 10^{18}$ GeV). In the code, field values are explicitly normalized by this (i.e. $x = \phi/M_{pl}$ implies $M_{pl}=1$ unit).
- **$S$**: A scaling parameter for **time** ($S = 5 \times 10^{-5}$ in this code). It relates physical time $t$ to code time $T$ via $dt = dT / (S \cdot M_{pl})$? Or rather $T$ is dimensionless time scaled by $S$. Actually, looking at $\dot{\phi} = S M_{pl} y$: $\frac{d\phi}{dt} = \frac{d(x M_{pl})}{d(T \cdot \text{something})} \dots$ See table below.

**Variable Mapping:**

| Code Variable | Exact Definition (Physical $\leftrightarrow$ Code) | Physical Meaning |
| :--- | :--- | :--- |
| **`x`** | $\phi / M_{pl}$ | Dimensionless Field Value |
| **`y`** | $\frac{d(\phi/M_{pl})}{dT} = \frac{\dot{\phi}}{S M_{pl}^2}$ (Assuming $t = T/S$?) | Dimensionless Field Velocity |
| **`z`** | $H / (S M_{pl})$ | Dimensionless Hubble Parameter |
| **`A`** | $a$ | Scale Factor (Dimensionless) |
| **`S`** | $S = 5 \times 10^{-5}$ | **Time Scaling Parameter** (Sets 'tick rate' of simulation) |
| **`M`** | $M = 5.9 \times 10^{-6}$ | **Mass Scale** (Simulated physical mass scale) |
| **`v0`** | $v_0 = 0.5 M^2$ (Quadratic) or $\lambda/(4\xi^2)$ (Higgs) | **Potential Amplitude** (Sets energy scale of inflation) |
| **`v`, `u`** | $\text{Re}(v_k), \text{Im}(v_k)$ | Scalar MS Variable (Dimension: Mass$^{-1/2}$ approx) |
| **`h`, `g`** | $\text{Re}(h_k), \text{Im}(h_k)$ | Tensor MS Variable |
| **`N`** | $\ln a$ | Number of e-folds |

## 2. Background Evolution & Pivot Scale Selection

We simulate the background evolution to find the end of inflation (defined where $\epsilon_H = 1$). We then identify the **pivot scale** $k_*$ as the mode that exits the horizon exactly $N_* = 60$ e-folds before the end.

$$ k_* = a(t_*) H(t_*) \quad \text{at} \quad N(t_*) = N_{\text{end}} - 60 $$

In [12]:
# Setup Model
# Use HiggsModel(xi=1000.0) to match the user's apparent context, or default
model = HiggsModel(xi=1000.0, lam=0.13)  
print(f"Model Initial Ai: {model.Ai}")
print(f"Model S: {model.S}")
print(f"Model xi: {model.xi_val}")

model.phi0= 7.0
model.yi = 0.0

# 1. Run Background
T_span_bg = np.linspace(0, 5000, 10000)
bg_sol = bg_solver.run_background_simulation(model, T_span_bg)
derived_bg = bg_solver.get_derived_quantities(bg_sol, model)

# 2. Find End of Inflation (epsH = 1)
epsH = derived_bg['epsH']
N = derived_bg['n'] # This is log(a)
N_efolds = derived_bg['N'] # Number of efolds since start

end_indices = np.where(epsH >= 1)[0]
if len(end_indices) > 0:
    end_idx = end_indices[0]
else:
    end_idx = -1
    print("Warning: Inflation did not end explicitly in the window.")

N_total = N_efolds[end_idx]
print(f"Inflation ends at N_sim = {N_total:.2f} (Index: {end_idx})")

# 3. Identify Pivot Scale (N_star = 60 before end)
N_star = 60.0
N_pivot = N_total - N_star
pivot_idx = np.argmin(np.abs(N_efolds - N_pivot))

# k = aH at pivot
x_bg, y_bg, z_bg, n_bg = bg_sol
a_pivot = np.exp(n_bg[pivot_idx])
H_pivot = z_bg[pivot_idx]
k_pivot = a_pivot * H_pivot

print(f"Pivot scale k* = {k_pivot:.4e}")
print(f"  -> Exits at Simulation N = {N_efolds[pivot_idx]:.2f}")
print(f"  -> This is {N_total - N_efolds[pivot_idx]:.2f} e-folds before the end of inflation.")
print(f"At pivot: a = {a_pivot:.4e}, H (z) = {H_pivot:.4e}")

Model Initial Ai: 1e-05
Model S: 5e-05
Model xi: 1000.0
Inflation ends at N_sim = 224.76 (Index: 220)
Pivot scale k* = 5.9266e+66
  -> Exits at Simulation N = 164.54
  -> This is 60.22 e-folds before the end of inflation.
At pivot: a = 2.8813e+66, H (z) = 2.0569e+00


## 3. Solving the Perturbations

Using the `run_ms_simulation` function from `inf_dyn_MS_full.py`, we evolve the perturbations for a specific $k$. 

**Initial Conditions**: We use **Bunch-Davies** initial conditions. We start the integration when the mode is deep inside the horizon ($k \gg aH$, typically $k \approx 100 aH$).
$$ v_k(\tau_{ini}) = \frac{1}{\sqrt{2k}} e^{-ik\tau} $$

The function returns the power spectra $P_{\mathcal{R}}$ (scalar) and $P_T$ (tensor) at the end of the simulation.

In [13]:
def calculate_for_mode(k_mode, model, bg_sol, end_idx):
    # Select start time when k ~ 100 aH (deep inside horizon)
    n_bg = bg_sol[3]
    z_bg = bg_sol[2]
    log_aH = n_bg + np.log(z_bg)
    target_start = np.log(k_mode) - np.log(100) 
    start_idx = np.argmin(np.abs(log_aH[:end_idx] - target_start))
    
    # Ensure we don't start too early if the simulation didn't go back that far
    start_idx = max(start_idx, 0)
    
    # Get ICs from background solution at this time
    xi = bg_sol[0][start_idx]
    yi = bg_sol[1][start_idx]
    zi = bg_sol[2][start_idx]
    Ai = np.exp(bg_sol[3][start_idx])
    
    # Time span: From start_idx to end_idx
    t_start = T_span_bg[start_idx]
    t_end = T_span_bg[end_idx]
    T_ms = np.linspace(t_start, t_end, 5000)
    
    # Run MS using the imported script function
    ms_sol = ms_solver.run_ms_simulation(xi, yi, zi, Ai, T_ms, k_mode, model)
    derived = ms_solver.get_ms_derived_quantities(ms_sol, model, k_mode)
    
    return derived['P_S'][-1], derived['P_T'][-1] # Return final values

## 4. Calculating Observables

We calculate the observables by comparing the power spectrum at the pivot scale $k_*$ and slightly shifted scales $k_{\pm} = k_* (1 \pm \delta)$.

**Spectral Index ($n_s$):**
$$ n_s = 1 + \frac{d \ln \mathcal{P}_{\mathcal{R}}}{d \ln k} \approx 1 + \frac{\ln \mathcal{P}(k_+) - \ln \mathcal{P}(k_-)}{\ln k_+ - \ln k_-} $$

**Tensor-to-Scalar Ratio ($r$):**
$$ r = \frac{\mathcal{P}_T(k_*)}{\mathcal{P}_{\mathcal{R}}(k_*)} $$

In [14]:
# Define k modes
delta = 0.01
ks = [k_pivot * (1 - delta), k_pivot, k_pivot * (1 + delta)]

results = []
print("Calculating power spectra...")
for k in ks:
    Ps, Pt = calculate_for_mode(k, model, bg_sol, end_idx)
    results.append((Ps, Pt))
    print(f"  k={k:.4e}: Ps={Ps:.4e}")

# Calculate ns
log_k = np.log(ks)
log_Ps = np.log([r[0] for r in results])

# Derivative d ln Ps / d ln k
slope = (log_Ps[2] - log_Ps[0]) / (log_k[2] - log_k[0])
ns = 1 + slope

# Calculate r at pivot (middle)
r_val = results[1][1] / results[1][0]

print("-"*30)
print(f"Results for {model.name}:")
print(f"n_s = {ns:.6f}")
print(f"r   = {r_val:.6f}")
print("-"*30)

Calculating power spectra...
  k=5.8673e+66: Ps=1.0684e+35
  k=5.9266e+66: Ps=1.7081e+35
  k=5.9859e+66: Ps=8.9271e+34
------------------------------
Results for Higgs Inflation:
n_s = -7.983970
r   = 0.002993
------------------------------


## 5. Saving Results

We save all relevant simulation data (model parameters, physical constants, initial conditions, and final observables) into a structured JSON file. This ensures reproducibility and ease of analysis.

In [15]:
def save_simulation_results(model, ns, r_val, k_pivot, N_total, N_pivot, results_list, ks_list, output_dir="../outputs/results"):
    """
    Saves simulation results in a structured, metadata-rich JSON format.
    """
    if not os.path.exists(output_dir):
        try:
            os.makedirs(output_dir)
        except OSError:
            pass # Handle race conditions or permission issues gracefully

    # Generate unique run ID
    run_id = str(uuid.uuid4())[:8]
    timestamp = datetime.datetime.now().isoformat()
    
    # Organize data
    data = {
        "metadata": {
            "run_id": run_id,
            "timestamp": timestamp,
            "description": "Exact numerical calculation of ns and r from Mukhanov-Sasaki equations."
        },
        "model_parameters": {
            "name": model.name,
            "xi": getattr(model, 'xi_val', None),
            "lambda": getattr(model, 'lam', None),
            "phi0": getattr(model, 'phi0', None),
            "yi": getattr(model, 'yi', None),
            "S_parameter": model.S,
            "potential_v0": model.v0
        },
        "background_info": {
            "total_efolds": float(N_total),
            "pivot_condition_N_star": 60.0
        },
        "pivot_scale": {
            "k_code_units": float(k_pivot),
            "exit_N_simulation": float(N_pivot)
        },
        "observables": {
            "n_s": float(ns),
            "r": float(r_val),
            "P_S_pivot": float(results_list[1][0]), # Middle value (pivot)
            "P_T_pivot": float(results_list[1][1])
        },
        "spectrum_scan": [
            {"k": float(k), "P_S": float(Ps), "P_T": float(Pt)} 
            for (k, (Ps, Pt)) in zip(ks_list, results_list)
        ]
    }
    
    # Construct detailed filename using formatted floats to avoid integer precision errors
    safe_name = model.name.replace(' ', '_').replace('(', '').replace(')', '')
    
    # Safely get variables, defaulting to 0 if not present, and ensure they are floats for formatting
    xi_val = float(getattr(model, 'xi_val', 0) or 0)
    lam_val = float(getattr(model, 'lam', 0) or 0)
    phi0_val = float(getattr(model, 'phi0', 0) or 0)
    yi_val = float(getattr(model, 'yi', 0) or 0)
    
    filename = f"{safe_name}_xi{xi_val:.1f}_lambda{lam_val:.1e}_phi0_{phi0_val:.1f}_yi_{yi_val:.3f}_run_{run_id}.json"
    filepath = os.path.join(output_dir, filename)
    
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=4)
        
    print(f"Results saved successfully to: {os.path.abspath(filepath)}")
    return filepath

# Call the function to save current results
save_simulation_results(model, ns, r_val, k_pivot, N_total, N_efolds[pivot_idx], results, ks)

Results saved successfully to: c:\Users\diego\OneDrive\Documentos\Universidad\Cosmologia\A-NumInflation\outputs\results\Higgs_Inflation_xi1000.0_lambda1.3e-01_phi0_7.0_yi_0.000_run_822f133c.json


'../outputs/results\\Higgs_Inflation_xi1000.0_lambda1.3e-01_phi0_7.0_yi_0.000_run_822f133c.json'